In [236]:
import numpy as np
import pandas as pd
import requests
from time import sleep

import sqlite3
import orjson


# Park type clustering based the venues nearby

Use FourSquare Place API to explore popular venues near the park (identified using latitude and longitude). The parks are then grouped based on similarity among their nearby venues.  

In [4]:
# load philly park info (philly park and state park)
conn = sqlite3.connect('philly_parks_info.db')
philly_parks = pd.read_sql_query("SELECT id, name, lat, lng FROM info", conn)
conn.close()

# philly_parks = philly_parks.set_index('id')

In [16]:
philly_parks.head()

,id,name,lat,lng
0,ChIJAWkAqNL1t4kRlm4slspOSXo,Quiet Waters Park,38.937670,-76.500899
1,ChIJzcow6Xb1t4kRQVE7s1AWWr8,Hillsmere Shores Community Beach,38.927212,-76.494150
2,ChIJ_bk5K1z1t4kRQNverIUOVko,Quiet Waters Dog Beach,38.930202,-76.508341
3,ChIJ_-2cQEX1t4kRoiCrmNMbrQ8,Quiet Waters Dog Park,38.931865,-76.505658
4,ChIJQWcsgIn2t4kRCL0Ub363FyI,Pip Moyer Recreation Center (Annapolis Recreat...,38.963271,-76.505150


In [11]:
# get fourSquare config ids
fsqr_config = pd.read_csv('foursquare_config.txt', header=None)
CLIENT_ID = fsqr_config.iloc[1][0]
CLIENT_SECRET = fsqr_config.iloc[3][0]
VERSION = '20200905' 
LIMIT = 500

In [37]:
# explore popular venues near the park location (ll)
def fsq_explore(ll):
    lat = ll[0]
    lng = ll[1]
    radius = 5000 # meters, max 100,000 meters; if 25000 m (about half hour drive at 30 mile/h)
    LIMIT = 500
    SECTION = ''
    # create the API request URL for venue recommendation
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}&time=any&day=any'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT, SECTION)

    # make the GET request
    results = requests.get(url).json()#["response"]['groups'][0]['items']
    return results

In [85]:
# example
results = fsq_explore(philly_parks.iloc[1][['lat', 'lng']])
print('total venues: ' + str(len(results['response']['groups'][0]['items'])))
for i in range(len(results['response']['groups'][0]['items'])):
    print(results['response']['groups'][0]['items'][i]['venue']['name'] + ': ' + 
         results['response']['groups'][0]['items'][i]['venue']['categories'][0]['name'] + 
         str(results['response']['groups'][0]['items'][i]['venue']['location']['distance']))

total venues: 35
Quiet Waters Park: Park1439
Bay Ridge Wine & Spirits: Wine Bar2450
Rocco's Pizza: Pizza Place2445
Annapolis Smokehouse and Tavern: BBQ Joint2469
Bark 'n' Bean: Pet Store2459
Caliente Grill: Mexican Restaurant2397
Main Ingredient: American Restaurant2451
Quiet Waters Dog Park: Dog Run1084
Oyster Harbor Beach: Beach2208
Old Stein Inn: German Restaurant3793
Vin909: Wine Bar4435
Bakers & co.: Bakery4687
Bruster's Real Ice Cream: Ice Cream Shop4076
Davis' Pub: Pub4818
Sammy's Italian Pizza Kitchen: Italian Restaurant4254
Sweet Hearts Patisserie: Bakery4032
Bayside Inn: Restaurant4283
Annapolis Seafood Markets: Seafood Restaurant3362
Grapes: Wine Bar4010
Historic London Town and Gardens: Garden4338
Wine Cellars of Annapolis: Wine Shop4016
Zumiez: Clothing Store3149
Triton Beach: Beach4868
"Pip" Moyer Recreation Center: Gym4094
Redbox: Video Store4262
7-Eleven: Convenience Store4267
Annapolis wine and spirits: Liquor Store3330
Sam's Supermarket: Grocery Store3457
Panda Noodle

In [56]:
results['response']['groups'][0]['items'][1]['venue']['categories']

[{'id': '4bf58dd8d48988d123941735',
  'name': 'Wine Bar',
  'pluralName': 'Wine Bars',
  'shortName': 'Wine Bar',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/winery_',
   'suffix': '.png'},
  'primary': True}]

In [112]:
nearby_venues_query('nyc_parks_info.db', 'nyc_park_nearby_venues_raw.db')

In [111]:
# philly_parks[['lat', 'lng']].apply(fsq_explore, axis=1)
# save query results in database

def nearby_venues_query(input_db, output_db):
    # input_db = 'philly_parks_info.db'; output_db = 'philly_park_nearby_venues_raw.db'
    in_conn = sqlite3.connect(input_db)
    out_conn = sqlite3.connect(output_db)
    out_conn.execute('''CREATE TABLE nearby_venues_raw (id text, name text, lat real, lng real, results blob)''')

    for ind, row in enumerate(in_conn.execute("SELECT id, name, lat, lng FROM info")):
        sleep(0.5)

        results = fsq_explore(row[2:])

        # insert place_details, curr_popularity, has_popular_times
        to_insert_values = [(row[0], row[1], row[2], row[3], orjson.dumps(results))]
        out_conn.executemany('''INSERT INTO nearby_venues_raw VALUES (?, ?, ?, ?, ?)''', to_insert_values)

        out_conn.commit()

    in_conn.close()
    out_conn.close()

In [109]:
conn = sqlite3.connect('philly_park_nearby_venues_raw.db')
test = pd.read_sql_query("SELECT * FROM nearby_venues_raw", conn)
conn.close()
test.head()

,id,name,lat,lng,results
0,ChIJAWkAqNL1t4kRlm4slspOSXo,Quiet Waters Park,38.937670,-76.500899,"b'{""meta"":{""code"":200,""requestId"":""606776a0225..."
1,ChIJzcow6Xb1t4kRQVE7s1AWWr8,Hillsmere Shores Community Beach,38.927212,-76.494150,"b'{""meta"":{""code"":200,""requestId"":""606776a1b26..."
2,ChIJ_bk5K1z1t4kRQNverIUOVko,Quiet Waters Dog Beach,38.930202,-76.508341,"b'{""meta"":{""code"":200,""requestId"":""606776a2808..."


In [138]:
park_ind = 1
nearby_venues = orjson.loads(test.iloc[park_ind]['results'])['response']['groups'][0]['items']
nearby_venues_name_distance = {nearby_venues[i]['venue']['name']:nearby_venues[i]['venue']['location']['distance'] for i in range(len(nearby_venues))}
nearby_venues_name_distance = {k: v for k, v in sorted(nearby_venues_name_distance.items(), 
                                                       key=lambda item: item[1], reverse=False)} # sort by distance
print(test.iloc[park_ind]['name'])
nearby_venues_name_distance

Hillsmere Shores Community Beach


{'Quiet Waters Dog Park': 1084,
 'Quiet Waters Park': 1439,
 'Oyster Harbor Beach': 2208,
 'Caliente Grill': 2397,
 "Rocco's Pizza": 2445,
 'Bay Ridge Wine & Spirits': 2450,
 'Main Ingredient': 2451,
 "Bark 'n' Bean": 2459,
 'Annapolis Smokehouse and Tavern': 2469,
 'Zumiez': 3149,
 'Annapolis wine and spirits': 3330,
 'Annapolis Seafood Markets': 3362,
 "Sam's Supermarket": 3457,
 'Old Stein Inn': 3793,
 'Panda Noodle House': 3997,
 'American Legion - Post 226': 3998,
 'Wild bird center': 3999,
 'Grapes': 4010,
 'Wine Cellars of Annapolis': 4016,
 'Sweet Hearts Patisserie': 4032,
 "Bruster's Real Ice Cream": 4076,
 '"Pip" Moyer Recreation Center': 4094,
 'Pit Boys': 4232,
 "Sammy's Italian Pizza Kitchen": 4254,
 'Redbox': 4262,
 '7-Eleven': 4267,
 'Bayside Inn': 4283,
 'Historic London Town and Gardens': 4338,
 'Vin909': 4435,
 'Sues Deli': 4509,
 'Bently Boat Slips': 4666,
 'Bakers & co.': 4687,
 'Rhode River Marina & Boat Sales': 4721,
 "Davis' Pub": 4818,
 'Triton Beach': 4868}

#### venue categories - convert to the top category (10 categ in total)

In [ ]:
# Build a dictionary for all possible fsq category ids, and their value will be the corresponding top category.

url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)
all_categ = requests.get(url).json()

In [264]:
# find the top level category for each categ id
# features are simplified by only considering the 10 top categ
#
# all_categ saved as a tree, with parent categ contains key-'categories', until reaching a leaf
def create_categ_mapping(all_categ):    
    categ_mapping = {}
    categ_parent = {}
    
    def find_categ(node, root_label, parent):
        # map id to label
        categ_mapping[node['id']] = root_label
        categ_parent[node['id']] = parent

        for child in node.get('categories', []): # if at leaf, categories is empty
            find_categ(child, root_label, node)  # 'node' is current child's parent; 'root_label' 1 of 10 top categ
        
            
    for itopc in range(10):
        root = all_categ['response']['categories'][itopc]
        find_categ(root, root['name'], None)
        
    # categ_mapping contains the 10 top categ corresponding to each id (np.unique(list(categ_mapping.values())))
    # categ_parent: direct parent of each categ_id (categ_parent['id']['categories'] are this parent's children) 
    return categ_mapping, categ_parent

categ_mapping, categ_parent = create_categ_mapping(all_categ)

In [282]:
categ_keys = np.unique(list(categ_mapping.values()))
# for itopc in range(10):
#     categ_keys.append(all_categ['response']['categories'][itopc]['name'])
categ_keys

array(['Arts & Entertainment', 'College & University', 'Event', 'Food',
       'Nightlife Spot', 'Outdoors & Recreation',
       'Professional & Other Places', 'Residence', 'Shop & Service',
       'Travel & Transport'], dtype='<U27')

#### extract feature - number of nearby venues per top categ

In [58]:

venue_stat = { k: 0 for k in categ_keys }

for item in results['response']['groups'][0]['items']:
    venue_top_categ = categ_mapping[item['venue']['categories'][0]['id']]
    venue_stat[venue_top_categ] += 1    




In [59]:
venue_stat

{'Arts & Entertainment': 0,
 'College & University': 0,
 'Event': 0,
 'Food': 14,
 'Nightlife Spot': 4,
 'Outdoors & Recreation': 8,
 'Professional & Other Places': 1,
 'Residence': 0,
 'Shop & Service': 8,
 'Travel & Transport': 0}

#### Foursquare sub-category id [detailed type info for each venue]

https://developer.foursquare.com/docs/build-with-foursquare/categories/), use search for the purpose of matching. 

In [140]:
# search the park in foursquare place API
def fsq_search(name, lat, lng):
    # create the API request URL for park search
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&name={}&intent=match'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, name)

    # make the GET request
    results = requests.get(url).json()#["response"]['groups'][0]['items']
    return results

In [141]:
def fsq_park_query(input_db, output_db):
    # input_db = 'philly_parks_info.db'; output_db = 'philly_park_nearby_venues_raw.db'
    in_conn = sqlite3.connect(input_db)
    out_conn = sqlite3.connect(output_db)
    out_conn.execute('''CREATE TABLE nearby_venues_raw (id text, name text, lat real, lng real, results blob)''')

    for ind, row in enumerate(in_conn.execute("SELECT id, name, lat, lng FROM info")):
        sleep(0.5)

        results = fsq_search(row[1], row[2], row[3])

        # insert place_details, curr_popularity, has_popular_times
        to_insert_values = [(row[0], row[1], row[2], row[3], orjson.dumps(results))]
        out_conn.executemany('''INSERT INTO nearby_venues_raw VALUES (?, ?, ?, ?, ?)''', to_insert_values)

        out_conn.commit()

    in_conn.close()
    out_conn.close()

In [143]:
fsq_park_query('philly_parks_info.db', "philly_park_foursquare_search.db")

fsq_park_query('nyc_parks_info.db', "nyc_park_foursquare_search.db")

#### Add new columns related to 'fsq_type' into info table

In [217]:
conn = sqlite3.connect("philly_park_foursquare_search.db")
fsq_park = pd.read_sql_query("SELECT * FROM nearby_venues_raw", conn)
conn.close()
fsq_park.head()

,id,name,lat,lng,results
0,ChIJAWkAqNL1t4kRlm4slspOSXo,Quiet Waters Park,38.937670,-76.500899,"b'{""meta"":{""code"":200,""requestId"":""6067a7d484d..."
1,ChIJzcow6Xb1t4kRQVE7s1AWWr8,Hillsmere Shores Community Beach,38.927212,-76.494150,"b'{""meta"":{""code"":200,""requestId"":""6067a7d54ba..."
2,ChIJ_bk5K1z1t4kRQNverIUOVko,Quiet Waters Dog Beach,38.930202,-76.508341,"b'{""meta"":{""code"":200,""requestId"":""6067a7d61c7..."
3,ChIJ_-2cQEX1t4kRoiCrmNMbrQ8,Quiet Waters Dog Park,38.931865,-76.505658,"b'{""meta"":{""code"":200,""requestId"":""6067a7d686e..."
4,ChIJQWcsgIn2t4kRCL0Ub363FyI,Pip Moyer Recreation Center (Annapolis Recreat...,38.963271,-76.505150,"b'{""meta"":{""code"":200,""requestId"":""6067a7d741d..."


In [228]:
# fsq_name, fsq_type_name, fsq_type_id

#fsq_park_results = fsq_park.iloc[ipark]['results']
def extract_fsq_park_info(fsq_park_results):
    response = orjson.loads(fsq_park_results)['response']
    if response and response['venues']: 
        venue = response['venues'][0]
        category = venue['categories'][0] if venue['categories'] else {}
        
        return pd.Series([venue['name'], category.get('name',''), category.get('id','')])
    else:
        return pd.Series(['','',''])

In [241]:
fsq_park[['fsq_name', 'fsq_type_name', 'fsq_type_id']] = fsq_park['results'].apply(extract_fsq_park_info)

fsq_park_light = fsq_park.drop(columns=['results'])
fsq_park_light

,id,name,lat,lng,fsq_name,fsq_type_name,fsq_type_id
0,ChIJAWkAqNL1t4kRlm4slspOSXo,Quiet Waters Park,38.937670,-76.500899,Quiet Waters Park,Park,4bf58dd8d48988d163941735
1,ChIJzcow6Xb1t4kRQVE7s1AWWr8,Hillsmere Shores Community Beach,38.927212,-76.494150,Hillsmere Shores Beach and Playground,Playground,4bf58dd8d48988d1e7941735
2,ChIJ_bk5K1z1t4kRQNverIUOVko,Quiet Waters Dog Beach,38.930202,-76.508341,,,
3,ChIJ_-2cQEX1t4kRoiCrmNMbrQ8,Quiet Waters Dog Park,38.931865,-76.505658,Quiet Waters Dog Park,Dog Run,4bf58dd8d48988d1e5941735
4,ChIJQWcsgIn2t4kRCL0Ub363FyI,Pip Moyer Recreation Center (Annapolis Recreat...,38.963271,-76.505150,"""Pip"" Moyer Recreation Center",Gym,4bf58dd8d48988d176941735
...,...,...,...,...,...,...,...
1651,ChIJfVzC4e1EwYkR7CX7hHL8w-I,Township Park,40.071413,-74.715783,,,
1652,ChIJN_dm4uVbwYkRmsPTaQhpCsI,Abbott Marshlands,40.163117,-74.702247,,,
1653,ChIJd9ol6rtbwYkR6v7KxxO7Z6k,Scenic Overlook,40.154833,-74.717484,Scenic Overlook,Scenic Lookout,4bf58dd8d48988d165941735
1654,ChIJ8VFM3PLjw4kRWHo927zq9ZM,Carson Road Woods,40.330333,-74.712685,,,


In [243]:
# save
fsq_park_light.to_parquet('philly_park_foursquare_types.parquet')

In [246]:
def add_fsq_type(input_db, output_parquet):
    conn = sqlite3.connect(input_db)
    fsq_park = pd.read_sql_query("SELECT * FROM nearby_venues_raw", conn)
    conn.close()
    
    # extract type info from results and add new columns
    fsq_park[['fsq_name', 'fsq_type_name', 'fsq_type_id']] = fsq_park['results'].apply(extract_fsq_park_info)
    
    fsq_park.drop(columns=['results']).to_parquet(output_parquet)


In [247]:
add_fsq_type('nyc_park_foursquare_search.db', 'nyc_park_foursquare_types.parquet')

#### Simplify the category to only one level below 'Outdoors & Recreation'
Use categ_parent

In [334]:
fsq_park = pd.read_parquet('nyc_park_foursquare_types.parquet')
fsq_park.head()

,id,name,lat,lng,fsq_name,fsq_type_name,fsq_type_id
0,ChIJDbzqHhXFw4kR5dCbhMQVy7Q,Dideriksen Park,40.402741,-74.430946,Dideriksen Park,Playground,4bf58dd8d48988d1e7941735
1,ChIJO5COJz3Fw4kRQxVRafcYWpw,East Brunswick Dog Park,40.400022,-74.440014,East Brunswick Dog Park,Dog Run,4bf58dd8d48988d1e5941735
2,ChIJM29CK8PFw4kRWZIbxcestzo,Gloria Bradford Borough Park,40.450825,-74.432411,Milltown's Borough Park,Baseball Field,4bf58dd8d48988d1e8941735
3,ChIJnQ9L1fPFw4kRZ7FelqVzLP4,Country Lane Park,40.443431,-74.414823,,,
4,ChIJKw0kXxHGw4kRn0bEGob21as,Rutgers Gardens,40.473419,-74.422737,Rutgers Gardens,Garden,4bf58dd8d48988d15a941735


In [310]:
# if top categ is 'Outdoors & Recreation', make sure it's only one level below 'Outdoors & Recreation' 
# (unless it's already 'Outdoors & Recreation')

def simplify_fsq_type(type_id_name):
    top_categ_name="Outdoors & Recreation"
    type_id, type_name = type_id_name
    if type_id:
        simplified_type = categ_mapping[type_id]
        if categ_mapping[type_id] == top_categ_name and type_name != top_categ_name:
            curr_id = type_id
            simplified_type = type_name

            while categ_parent[curr_id]['name'] != top_categ_name:
                simplified_type = categ_parent[curr_id]['name']
                curr_id = categ_parent[curr_id]['id']

        return simplified_type


In [335]:
fsq_park['fsq_type_name_simplified'] = fsq_park[['fsq_type_id', 'fsq_type_name']].apply(simplify_fsq_type, axis=1)

In [341]:
fsq_park.groupby('fsq_type_name_simplified').size().sum()

2100

In [337]:
fsq_park.to_parquet('nyc_park_foursquare_types.parquet')

In [340]:
df = pd.read_parquet('philly_park_foursquare_types.parquet')
df.groupby('fsq_type_name_simplified').size().sum()

1046

In [ ]:
# TODO:
# other type feature - whether it is a state park
